<a href="https://colab.research.google.com/github/Rohithkr02/nlpproject/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV  #finding best model
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/redditpreprocess.csv')


In [ ]:
df.head()

,post_id,text,label_adr
0,8zr5u3,Ive always chewed nail never knew anxiety alwa...,withdraw
1,8zr5u3_aug,Ive always chewed nail never knew anxiety alwa...,withdraw
2,8zr5u3_aug,Ive always chewed nail never knew anxiety alwa...,withdraw
3,8zr5u3_aug,Ive always chewed nail never knew anxiety anxi...,withdraw
4,8zgg5h,Ive Effexor XR 8 year started 75mg 4 year ago ...,withdraw


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label_adr'] = le.fit_transform(df['label_adr'])

In [ ]:
# For BERT:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

## Want DistilBERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
tokenized = df["text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,truncation=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(756, 512)

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
print(attention_mask.size())

torch.Size([756, 512])


In [ ]:
labels = df["label_adr"]
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,ConfusionMatrixDisplay,accuracy_score
rr=SVC()
rd=DecisionTreeClassifier()
rf=RandomForestClassifier()
lst=[rr,rd,rf]
for i in lst:
  i.fit(X_train,y_train)
  y_pred1=i.predict(X_test)
  print('**************')
  print('**************')
  print(classification_report(y_test,y_pred1))
  print('**************')
  print('**************')

**************
**************
              precision    recall  f1-score   support

           0       0.50      0.12      0.19        17
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00        24
           3       0.50      0.11      0.18        18
           4       0.00      0.00      0.00        14
           5       0.00      0.00      0.00        11
           6       1.00      0.06      0.11        18
           7       0.37      0.97      0.54        69

    accuracy                           0.38       189
   macro avg       0.30      0.16      0.13       189
weighted avg       0.32      0.38      0.24       189

**************
**************
**************
**************
              precision    recall  f1-score   support

           0       0.38      0.59      0.47        17
           1       0.17      0.22      0.19        18
           2       0.63      0.50      0.56        24
           3       0.43      0.56      0.4